# Discovery API Test

## APIの初期化

In [1]:
# 必要ライブラリの導入
!pip install ibm_watson | tail -n 1

In [2]:
# credential情報 (個別に設定します)

discovery_credentials = {
  "apikey": "xxxx",
  "url": "https://gateway.watsonplatform.net/discovery/api"
}

In [3]:
# Discovery APIの初期化

import json
import os
from ibm_watson import DiscoveryV1

version = '2019-04-30'

discovery = DiscoveryV1(
    version=version,
    iam_apikey=discovery_credentials['apikey'],
    url=discovery_credentials['url']
)

In [4]:
# environment_id、collection_id、configuration_id の取得
# すでにUIで1つのprivate collectionが作成済みであることが前提

# environment id の取得
environment_id = discovery.list_environments().get_result()['environments'][1]['environment_id']
print('environment_id: ', environment_id)

# collection id の取得
collection_id = discovery.list_collections(environment_id ).get_result()['collections'][1]['collection_id']
print('collection_id: ' , collection_id)

# configuration_idの取得
configuration_id = discovery.list_configurations(environment_id).get_result()['configurations'][0]['configuration_id']
print('configuration_id: ', configuration_id)

environment_id:  2c134ad3-42c4-48fe-bf84-3731d0f8cfe3
collection_id:  601acd99-511a-4b2a-bda7-c6c8f4f35ad4
configuration_id:  1a09551f-bf55-4d4a-9741-b2612e5e61fd


## 文書のロードと削除

In [5]:
# 文書ロード関数
# collection_id: 対象コレクション
# sample_data: 書き込み対象テキスト (json形式の配列)
# key_name: 文書のユニークキー名称

def load_text( collection_id, sample_data, key_name):
    for item in sample_data:
        
        # itemごとにワークのjsonファイルを作成
        print(item)
        key = item.get(key_name)
        filename = str(key) + '.json'
        f = open(filename, 'w')
        json.dump(item, f)
        f.close()
        
        # 書き込み可能かのチェック
        collection = discovery.get_collection(environment_id, collection_id).get_result()
        proc_docs = collection['document_counts']['processing']
        while True:
            if proc_docs < 20:
                break
            print('busy. waiting..')
            time.sleep(10)
            collection = discovery.get_collection(environment_id, collection_id)
            proc_docs = collection['document_counts']['processing']

        # jsonファイル名を引数にDiscoveryへデータロード
        with open(filename) as f:
            add_doc = discovery.add_document(environment_id, collection_id, file = f)
        os.remove(filename)

In [6]:
# 特定のコレクションの全文書を削除する関数
# collection_id: 対象コレクション

def delete_all_docs(collection_id):

    # 文書件数取得
    collection = discovery.get_collection(environment_id, collection_id).get_result()
    doc_count = collection['document_counts']['available']

    results = discovery.query(environment_id, collection_id, return_fields='id', count=doc_count).get_result()["results"]
    ids = [item["id"] for item in results]

    for id in ids:
        print('deleting doc: id =' + id)
        discovery.delete_document(environment_id, collection_id, id)

In [7]:
# ロードテスト用テキスト
sample_data = [
    {'app_id': 1, 'title': '最初のテキスト', 'text': 'サンプルテキストその1。'},
    {'app_id': 2, 'title': '2番目のテキスト', 'text': '新幹線はやぶさが好きです。'},
    {'app_id': 3, 'title': '3番目のテキスト', 'text': '令和元年に転職しました。'},
]

In [9]:
# 文書ロードテスト
load_text(collection_id, sample_data, 'app_id')

{'app_id': 1, 'title': '最初のテキスト', 'text': 'サンプルテキストその1。'}
{'app_id': 2, 'title': '2番目のテキスト', 'text': '新幹線はやぶさが好きです。'}
{'app_id': 3, 'title': '3番目のテキスト', 'text': '令和元年に転職しました。'}


In [8]:
# 全件削除テスト
delete_all_docs(collection_id)

deleting doc: id =095023d6-7b9f-4fc9-8cae-220f0dac5b64
deleting doc: id =a2d805d8-ac39-4f13-adbf-ebdf60f214b0
deleting doc: id =53cb408b-abc6-4fe0-8fa2-5753251f4cef


## 文書検索

In [10]:
# 検索用関数
# collection_id: 検索対象コレクション
# query_text: 検索条件式
# return_fields: 出力項目

def query_documents(collection_id, query_text, return_fields):
    # 文書件数取得
    collection = discovery.get_collection(environment_id, collection_id).get_result()
    doc_count = collection['document_counts']['available']
    print('doc_count: ', doc_count)

    query_results = discovery.query(environment_id, collection_id, 
        query=query_text, 
        count=doc_count, 
        return_fields=return_fields).get_result()[ "results"]
    return query_results

### textフィールドに「サンプル」を含む文書の検索

In [11]:
query_text = 'text:サンプル'
return_fields = 'app_id,title,text'
query_results = query_documents(collection_id, query_text, return_fields)

print(json.dumps(query_results, indent=2, ensure_ascii=False))

doc_count:  2
[
  {
    "id": "965f5a5a-2bd1-4118-9517-3c187c47a02c",
    "result_metadata": {
      "confidence": 0.08408801890816446,
      "score": 1.0226655
    },
    "text": "サンプルテキストその1。",
    "title": "最初のテキスト",
    "app_id": 1
  }
]
